#### 1. DB 연결하기

In [1]:
from dotenv import load_dotenv
import os
import psycopg2
import io
import polars as pl

In [2]:
load_dotenv('.env')
db_name = os.getenv('db_name')
user_name = os.getenv('db_admin')
password = os.getenv('password')

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database=db_name,
    user=user_name,
    password=password
)

cur = conn.cursor()
cur.execute("select now();")
print(cur.fetchone())

(datetime.datetime(2026, 1, 22, 14, 16, 28, 965266, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),)


In [4]:
from datetime import datetime
import pandas as pd

now = datetime(2110,2,4,16,17,30)

patients_list = pd.read_sql(
    f"""
    SELECT *
    FROM patients
    WHERE
        intime >= TIMESTAMP '{str(now)}'
    AND intime <= TIMESTAMP '{str(now)}' + INTERVAL '24 hours'  -- 앞으로 24시간
    AND (
        outtime IS NULL
        OR outtime > TIMESTAMP '{str(now)}'
    )
    ORDER BY intime;
""", conn)

patients_list

C:\Users\amysm\AppData\Local\Temp\ipykernel_11300\2087530985.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patients_list = pd.read_sql(


,subject_id,hadm_id,stay_id,intime,outtime,intime24,gender,환자실제나이,dod_within_24h,최초입실
0,12963969,22651481,32470270,2110-02-04 16:17:30,2110-02-07 17:41:58,2110-02-05 16:17:30,M,79,0,1
1,13763968,26290668,31613027,2110-02-04 18:22:54,2110-02-07 07:33:17,2110-02-05 18:22:54,M,82,0,1


#### 2. parquet -> csv 변환

In [ ]:
data_path = 'C:\mini_postgre\\for_streamlit_data\\'

patient = pl.scan_parquet(data_path + 'SICU_patients.parquet')

patient = patient.drop(
    ["los","dod",'anchor_age','anchor_year','anchor_year_group','edregtime','edouttime',
    'admittime','dischtime','deathtime','admission_type','admit_provider_id',
    'admission_location','discharge_location','insurance','language','marital_status',
    'race','edregtime','edouttime','hospital_expire_flag','anchor_age','anchor_year',
    'anchor_year_group','dod',])

patient = patient.with_columns(
    pl.col("intime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.col("outtime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.col("intime24").str.to_datetime("%Y-%m-%d %H:%M:%S")
)

patient = patient.collect().to_pandas()
buffer = io.StringIO()
patient.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.patients
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

In [ ]:
input = pl.scan_parquet(data_path + 'SICU_inputevents.parquet')

input = (
    input
    # 1. 컬럼 추가
    .with_columns([
        pl.lit("inputevents").alias("tablename"),
        pl.lit(None).alias("value"),
    ])
    # 2. 컬럼명 변경
    .rename({
        "amount": "valuenum",
        "amountuom": "valueuom",
    })
    # 3. 컬럼 선택 + 순서 정렬
    .select(["subject_id","hadm_id","stay_id","starttime","endtime",
             "itemid","label","value","valuenum","valueuom","tablename",]))

input = input.collect().to_pandas()
buffer = io.StringIO()
input.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.inputevents
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

In [41]:
output = pl.scan_parquet(data_path + 'SICU_outputevents.parquet')

import polars as pl

output = (
    output
    .with_columns([
        pl.col("value").alias("valuenum"),          
        pl.lit("outputevents").alias("tablename"),  
        pl.lit(None).cast(pl.Datetime).alias("endtime"),
    ])
    # 3. 컬럼명 변경
    .rename({
        "charttime": "starttime",
    })
    # 4. 컬럼 순서 정렬
    .select(["subject_id","hadm_id","stay_id","starttime","endtime",
             "itemid","label","value","valuenum","valueuom","tablename",]))

output = output.collect().to_pandas()
buffer = io.StringIO()
output.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.outputevents
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

In [45]:
def return_lab(lab):
    lab = lab[['subject_id','hadm_id','stay_id','charttime',
           'itemid','label','value','valuenum','valueuom']]
    lab['table'] = 'labevents'
    lab['endtime'] = None
    lab = lab.rename(columns={'charttime':'starttime',})
    lab = lab[['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'label', 'value',
                'valuenum', 'valueuom', 'table']]
    return lab

In [59]:
lab = pl.scan_parquet(data_path + 'SICU_labevents.parquet')

lab = (
    lab
    # 1. 컬럼 추가
    .with_columns([
        pl.lit("labevents").alias("tablename"),
        pl.lit(None).alias("endtime"),
    ])
    # 2. 컬럼명 변경
    .rename({
        "charttime":"starttime",
    })
    # 3. 컬럼 선택 + 순서 정렬
    .select(["subject_id","hadm_id","stay_id","starttime","endtime",
             "itemid","label","value","valuenum","valueuom","tablename"]))

lab = lab.collect().to_pandas()
buffer = io.StringIO()
lab['hadm_id'] = lab['hadm_id'].astype('Int64')
lab.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.labevents
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

lab.shape

(3200785, 11)

In [60]:
chart = pl.scan_parquet(data_path + 'SICU_chartevents.parquet')

chart = (
    chart
    # 1. 컬럼 추가
    .with_columns([
        pl.lit("chartevents").alias("tablename"),
        pl.lit(None).alias("endtime"),
    ])
    # 2. 컬럼명 변경
    .rename({
        "charttime":"starttime",
    })
    # 3. 컬럼 선택 + 순서 정렬
    .select(["subject_id","hadm_id","stay_id","starttime","endtime",
             "itemid","label","value","valuenum","valueuom","tablename"]))

chart = chart.collect().to_pandas()
buffer = io.StringIO()
chart['hadm_id'] = chart['hadm_id'].astype('Int64')
chart.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.chartevents
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

chart.shape

(49185598, 11)

In [64]:
diag = pl.scan_parquet(data_path + 'diagnoses_icd.parquet')
diag = diag.collect().to_pandas()
buffer = io.StringIO()
diag.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.diagnoses_icd
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

diag.shape

(6364488, 5)

In [66]:
pro = pl.scan_parquet(data_path + 'procedures_icd.parquet')
pro = pro.collect().to_pandas()
buffer = io.StringIO()
pro.to_csv(buffer, index=False, header=True)
buffer.seek(0)

cur.copy_expert(
    """
    COPY sicu.procedures_icd
    FROM STDIN
    WITH CSV HEADER
    """,
    buffer
)

conn.commit()

pro.shape

(859655, 6)

In [3]:
import pandas as pd

cur = conn.cursor()
patient = pd.read_sql(f"select * from patients where stay_id = 31488097;", conn)
patient

C:\Users\amysm\AppData\Local\Temp\ipykernel_20864\807463521.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patient = pd.read_sql(f"select * from patients where stay_id = 31488097;", conn)


,subject_id,hadm_id,stay_id,intime,outtime,intime24,gender,환자실제나이,dod_within_24h,최초입실
0,18727857,24276406,31488097,2176-02-12 00:07:20,2176-02-18 15:40:14,2176-02-13 00:07:20,M,39,0,0


In [8]:
from datetime import datetime
from streamlit_dashboard_func import * 

now = datetime(2110,2,4,16,17,30)
selected_patient = 31488097

input = stay_filtering('inputevents', selected_patient)
output = stay_filtering('outputevents', selected_patient)
lab = stay_filtering('labevents', selected_patient)
chart = stay_filtering('chartevents', selected_patient)

results = pd.concat([input, output, lab, chart])
results

C:\Users\amysm\AppData\Local\Temp\ipykernel_20864\1252203590.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([input, output, lab, chart])


,subject_id,hadm_id,stay_id,starttime,endtime,itemid,label,value,valuenum,valueuom,tablename
0,18727857,24276406,31488097,2176-02-12 03:42:00,2176-02-12 03:49:00,225943,Solution,None,3.287162,mL,inputevents
1,18727857,24276406,31488097,2176-02-12 03:45:00,2176-02-12 06:24:00,220949,Dextrose 5%,None,100.668241,mL,inputevents
2,18727857,24276406,31488097,2176-02-12 03:45:00,2176-02-12 06:24:00,221749,Phenylephrine,None,24.160376,mg,inputevents
3,18727857,24276406,31488097,2176-02-12 03:49:00,2176-02-12 05:59:00,222168,Propofol,None,839.080044,mg,inputevents
4,18727857,24276406,31488097,2176-02-12 03:49:00,2176-02-12 05:59:00,225943,Solution,None,83.908011,mL,inputevents
...,...,...,...,...,...,...,...,...,...,...,...
4202,18727857,24276406,31488097,2176-02-12 23:00:00,NaT,220050,Arterial Blood Pressure systolic,88,88.000000,mmHg,chartevents
4203,18727857,24276406,31488097,2176-02-12 23:00:00,NaT,224650,Ectopy Type 1,None,NaN,None,chartevents
4204,18727857,24276406,31488097,2176-02-13 00:00:00,NaT,226169,Subglottal Suctioning,1,1.000000,None,chartevents
4205,18727857,24276406,31488097,2176-02-13 00:00:00,NaT,227951,Circulation/Skin Integrity,Circulation adequate,NaN,None,chartevents


In [ ]:
def already_predicted(cur, stay_id, hours): 
        sql = """
        SELECT 1
        FROM mortality_prediction
        WHERE stay_id = %s AND hours = %s
        LIMIT 1
        """
        cur.execute(sql, (stay_id, hours))
        return cur.fetchone() is not None